In [11]:
from collections import deque
from queue import PriorityQueue
from quality_measure import quality_measure
from load_data import make_growth_target_df
import pandas as pd
import numpy as np
import math

## aantekeningen
\
windowsize is erg bepalend in welke subgroups de hoogste quality measure halen (max 72 vs 250 bij 7 vs 5 windowsize) \
\
windowsize, windowoverlap en aggregate functies moeten nog als input aan beam search toegevoegd worden (voor de quality measure) \
\
we doen nu alleen equal voor classes column maar moet ook niet not equal? (dan neemt mogelijke optie wel gigantish toe (exponentieel??)) \
\
worden de splitpoints voor numerical class nu wel goed berekent en hoeveel bins is optimaal? \
\
is equalsized bins de beste optie? \
\
in de results opslaan hoe groot de subgroup is

In [31]:
def gteq(a, b):
    return a >= b

def leeq(a, b):
    return a <= b

def eq(a, b):
    return a == b

def neq(a, b):
    return not eq(a, b)

def extract_subgroup(descriptors, data):
    result = []
    for row in data:
        check = True
        for attribute in descriptors:
            att_index, descr_value, operator = attribute # unpack 3 values from attribute
            value = operator(row[att_index], descr_value)

            if not value:
                check = False
                break
        
        if check:
            result.append(row)

    return result


def refin(seed, data, types, nr_bins, descr_indices):
    print(len(types), len(descr_indices))
    res = []
    used_descr = [i[0] for i in seed]
    not_used_indices = descr_indices[:]
    [not_used_indices.remove(i) for i in used_descr]
    # refinement of descriptors

    for i in not_used_indices: # functies voor voorwaarden gewoon erin doen
        aux = list(seed)[:]

        if types[i] == 'numeric':
            s = extract_subgroup(seed, data)
            all_values = [float(entry[i]) for entry in s]
            all_values = sorted(all_values)
            n = len(all_values)
            split_points = [all_values[math.floor(j * (n/nr_bins))] for j in range(1, nr_bins)] # weet niet of dit nu goed gaat
            for s in split_points:
                func1 = leeq
                func2 = gteq
         
                local0 = aux[:]
                local0.append((i, s, func1))
                res.append(local0)
           
                local1 = aux[:]
                local1.append((i, s, func2))
                res.append(local1)

        elif types[i] == 'binary':
            func = eq
            local0 = aux[:]
            local0.append((i, 0, func))
            local1 = aux[:]
            local1.append((i, 1, func))
            res.append(local0)
            res.append(local1)

        else:
            all_values = [entry[i] for entry in data]
            for j in set(all_values):
                func1 = eq
                func2 = neq
                local0 = aux[:]
                local0.append((i, j, func1))
                res.append(local0)
    return res

            

def constraints_satisfied(descriptors, constraints): # and subgroup len > 0
    return True

def put_item_in_queue(queue, quality, descriptor):
    if queue.full():
        min_quality, min_descriptor = queue.get()
        if min_quality >= quality:
            queue.put((min_quality, min_descriptor))
        else:
            queue.put((quality, descriptor))
    else:
        queue.put((quality, descriptor))

def categorize_columns_in_order(df, att_columns):
    # Define empty list to store the categories in order
    column_types = []

    # Loop through attribute columns in the DataFrame
    for col in att_columns:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(df[col]):
            column_types.append('numeric')
        # Check if the column has exactly 2 unique values (binary)
        elif df[col].nunique() == 2:
            column_types.append('binary')
        # Otherwise, treat it as nominal
        else:
            column_types.append('nominal')

    return column_types


In [32]:
def beam_search(data, targets_baseline, column_names, quality, beam_width, beam_depth, nr_bins, nr_saved, subgroup_size, constraints, target, types):

    target_ind = column_names.index(target) 
    att_indices = list(range(0, len(column_names)))
    att_indices.remove(target_ind) # ONLY INCLUDE TARGET GROWTH!
    beam_queue = deque([()]) # initialize with emtpy tuple (the empty set)
    results = PriorityQueue(nr_saved) # queue with max amount of descriptors saved

    for depth in range(beam_depth):
        beam = PriorityQueue(beam_width) # amount of combinations we keep investigating
    
        while bool(beam_queue):# while there are items in the queue
            seed = beam_queue.popleft()
            descriptor_set = refin(seed, data, types, nr_bins, att_indices)

            for descriptor in descriptor_set:
                subgroup = extract_subgroup(descriptor, data)
                if constraints_satisfied(descriptor, constraints) and len(subgroup) >= subgroup_size:
                    targets_subgroup = [i[target_ind] for i in subgroup]
                    quality_result = quality(targets_subgroup, targets_baseline)
                    put_item_in_queue(results, quality_result, tuple(descriptor))
                    put_item_in_queue(beam, quality_result, tuple(descriptor))
        


        while not beam.empty(): 

            new_combination = beam.get()
            new_combination = new_combination[1]# dit moet het item gaan geven met de beste quality measure
            beam_queue.append(new_combination)

    return results



In [14]:
# Create stock_df from the target file
stock_df = make_growth_target_df('stock_data_for_emm.pkl')

In [33]:
stock_df.drop(['index'], inplace=True, axis=1)
stock_df

KeyError: "['index'] not found in axis"

In [34]:
data = stock_df.values.tolist()
column_names = list(stock_df.columns)
quality = quality_measure
beam_width = 3
beam_depth = 3
nr_bins = 3
nr_saved = 10
subgroup_size = 10
constraints = None
target = 'growth_target'

In [35]:
target_ind = column_names.index(target)
all_time_series = [i[target_ind] for i in data]
all_time_series = np.array(all_time_series)
targets_baseline = np.mean(all_time_series, axis=0)

In [36]:
att_indices = list(range(0, len(column_names)))
att_indices.remove(target_ind)
att_columns = [column_names[i] for i in att_indices]
types = categorize_columns_in_order(stock_df, att_columns)

In [37]:
results  = beam_search(data, targets_baseline, column_names, quality, beam_width, beam_depth, nr_bins, nr_saved, subgroup_size, constraints, target, types)


11 11


KeyboardInterrupt: 

In [20]:
while not results.empty():
    item = results.get()
    print(f"Priority: {item[0]}, Descriptor: {item[1]}")

Priority: 64.86614408246143, Descriptor: ((3, 'Europe/Berlin', <function eq at 0x00000231E5B49EA0>), (2, 'EUR', <function eq at 0x00000231E5B49EA0>))
Priority: 64.86614408246143, Descriptor: ((3, 'Europe/Berlin', <function eq at 0x00000231E5B49EA0>), (6, 0.0, <function gteq at 0x00000231E5D39FC0>))
Priority: 65.62197956465624, Descriptor: ((10, 10000.0, <function gteq at 0x00000231E5D39FC0>), (6, 39.0, <function gteq at 0x00000231E5D39FC0>))
Priority: 65.81396857128121, Descriptor: ((2, 'EUR', <function eq at 0x00000231E5B49EA0>), (6, 0.0, <function gteq at 0x00000231E5D39FC0>), (7, 5.111, <function gteq at 0x00000231E5D39FC0>))
Priority: 65.81396857128121, Descriptor: ((2, 'EUR', <function eq at 0x00000231E5B49EA0>), (7, 5.111, <function gteq at 0x00000231E5D39FC0>))
Priority: 67.74505877067688, Descriptor: ((10, 10000.0, <function gteq at 0x00000231E5D39FC0>), (8, 6693983744.0, <function gteq at 0x00000231E5D39FC0>))
Priority: 68.37178453583074, Descriptor: ((2, 'EUR', <function eq a